##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/tff_for_federated_learning_research_compression"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/tff_for_federated_learning_research_compression.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/tff_for_federated_learning_research_compression.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/federated/tutorials/tff_for_federated_learning_research_compression.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

# TFF para pesquisa de aprendizado federado – Compressão de modelo e atualização

**OBSERVAÇÃO**: foi verificado que este Colab funciona com a [versão mais recente lançada](https://github.com/tensorflow/federated#compatibility) do pacote pip `tensorflow_federated`. Talvez não seja possível atualizar este Colab para funcionar no `master`.

Neste tutorial, usamos o dataset [EMNIST](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/emnist) para demonstrar como permitir algoritmos de compressão com perda para reduzir o custo de comunicação no algoritmo de cálculo federado de médias usando a API `tff.learning`. Para mais detalhes sobre o algoritmo de cálculo federado de médias, confira o artigo [Communication-Efficient Learning of Deep Networks from Decentralized Data](https://arxiv.org/abs/1602.05629) (Aprendizado de redes profundas com comunicação eficiente usando dados descentralizados).

## Antes de começarmos

Antes de começarmos, execute o código abaixo para que o ambiente seja configurado corretamente. Se não for exibida uma saudação, consulte as instruções de [instalação](../install.md).

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade tensorflow-model-optimization

In [ ]:
%load_ext tensorboard

import functools

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

Verifique se o TFF está funcionando.

In [ ]:
@tff.federated_computation
def hello_world():
  return 'Hello, World!'

hello_world()

b'Hello, World!'

## Prepare os dados de entrada

Nesta seção, carregamos e pré-processamos o dataset EMNIST incluído no TFF. Confira mais detalhes sobre esse dataset no tutorial [Aprendizado federado para classificação de imagens](https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification#preparing_the_input_data).


In [ ]:
# This value only applies to EMNIST dataset, consider choosing appropriate
# values if switching to other datasets.
MAX_CLIENT_DATASET_SIZE = 418

CLIENT_EPOCHS_PER_ROUND = 1
CLIENT_BATCH_SIZE = 20
TEST_BATCH_SIZE = 500

emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(
    only_digits=True)

def reshape_emnist_element(element):
  return (tf.expand_dims(element['pixels'], axis=-1), element['label'])

def preprocess_train_dataset(dataset):
  """Preprocessing function for the EMNIST training dataset."""
  return (dataset
          # Shuffle according to the largest client dataset
          .shuffle(buffer_size=MAX_CLIENT_DATASET_SIZE)
          # Repeat to do multiple local epochs
          .repeat(CLIENT_EPOCHS_PER_ROUND)
          # Batch to a fixed client batch size
          .batch(CLIENT_BATCH_SIZE, drop_remainder=False)
          # Preprocessing step
          .map(reshape_emnist_element))

emnist_train = emnist_train.preprocess(preprocess_train_dataset)

## Definição do modelo

Definimos um modelo do Keras baseado na CNN FedAvg original e então encapsulamos o modelo do Keras em uma instância de [tff.learning.models.VariableModel](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model) para que ele possa ser consumido pelo TFF.

Precisaremos de uma **função** que gere um modelo em vez de simplesmente um modelo direto. Além disso, a função **não pode** simplesmente capturar um modelo pré-construído, ela precisa criar o modelo no contexto em que é chamada. O motivo para isso é que o TFF foi projetado para uso em dispositivos e precisa controlar quando os recursos são construídos para que eles possam ser capturados e empacotados.

In [ ]:
def create_original_fedavg_cnn_model(only_digits=True):
  """The CNN model used in https://arxiv.org/abs/1602.05629."""
  data_format = 'channels_last'

  max_pool = functools.partial(
      tf.keras.layers.MaxPooling2D,
      pool_size=(2, 2),
      padding='same',
      data_format=data_format)
  conv2d = functools.partial(
      tf.keras.layers.Conv2D,
      kernel_size=5,
      padding='same',
      data_format=data_format,
      activation=tf.nn.relu)

  model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
      conv2d(filters=32),
      max_pool(),
      conv2d(filters=64),
      max_pool(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation=tf.nn.relu),
      tf.keras.layers.Dense(10 if only_digits else 62),
      tf.keras.layers.Softmax(),
  ])

  return model

# Gets the type information of the input data. TFF is a strongly typed
# functional programming framework, and needs type information about inputs to 
# the model.
input_spec = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0]).element_spec

def tff_model_fn():
  keras_model = create_original_fedavg_cnn_model()
  return tff.learning.models.from_keras_model(
      keras_model=keras_model,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## Treinamento do modelo e geração das métricas de treinamento como saída

Agora está tudo pronto para construirmos o algoritmo de cálculo federado de médias e treinarmos o modelo definido com o dataset EMNIST.

Primeiro, precisamos construir um algoritmo de cálculo federado de médias usando a API [tff.learning.algorithms.build_weighted_fed_avg](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg).

In [ ]:
federated_averaging = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=tff_model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

Agora, vamos executar o algoritmo de cálculo federado de médias. Sua execução pela perspectiva do TFF é da seguinte forma:

1. Inicialize o algoritmo e obtenha o estado inicial do servidor, que contém as informações necessárias para executar o algoritmo. Lembre-se de que, como o TFF é funcional, esse estado inclui tanto o estado do otimizador usado pelo algoritmo (ou seja, termos do momento) quanto os parâmetros do modelo em si (que serão passados como argumentos e retornados como resultados a partir das computações do TFF).
2. Execute o algoritmo rodada por rodada. Em cada rodada, um novo estado do servidor será retornado como resultado do treinamento do modelo feito por cada cliente com seus dados. Tipicamente, em uma rodada:
    1. O servidor faz broadcast do modelo para todos os clientes participantes.
    2. Cada cliente realiza o trabalho com base no modelo e seus próprios dados.
    3. O servidor agrega todo o modelo para gerar um estado do servidor que contenha um novo modelo.

Confira mais detalhes no tutorial [Algoritmos federados personalizados, parte 2: Implementando o cálculo federado de médias](https://www.tensorflow.org/federated/tutorials/custom_federated_algorithms_2).

As métricas de treinamento são escritas no diretório do TensorBoard para exibição após o treinamento.

In [ ]:
def train(federated_averaging_process, num_rounds, num_clients_per_round, summary_writer):
  """Trains the federated averaging process and output metrics."""

  # Initialize the Federated Averaging algorithm to get the initial server state.
  state = federated_averaging_process.initialize()

  with summary_writer.as_default():
    for round_num in range(num_rounds):
      # Sample the clients parcitipated in this round.
      sampled_clients = np.random.choice(
          emnist_train.client_ids,
          size=num_clients_per_round,
          replace=False)
      # Create a list of `tf.Dataset` instances from the data of sampled clients.
      sampled_train_data = [
          emnist_train.create_tf_dataset_for_client(client)
          for client in sampled_clients
      ]
      # Round one round of the algorithm based on the server state and client data
      # and output the new state and metrics.
      result = federated_averaging_process.next(state, sampled_train_data)
      state = result.state
      train_metrics = result.metrics['client_work']['train']

      # Add metrics to Tensorboard.
      for name, value in train_metrics.items():
          tf.summary.scalar(name, value, step=round_num)
      summary_writer.flush()

In [ ]:
# Clean the log directory to avoid conflicts.
try:
  tf.io.gfile.rmtree('/tmp/logs/scalars')
except tf.errors.OpError as e:
  pass  # Path doesn't exist

# Set up the log directory and writer for Tensorboard.
logdir = "/tmp/logs/scalars/original/"
summary_writer = tf.summary.create_file_writer(logdir)

train(federated_averaging_process=federated_averaging, num_rounds=10,
      num_clients_per_round=10, summary_writer=summary_writer)

round  0, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.092454836), ('loss', 2.310193), ('num_examples', 941), ('num_batches', 51)]), broadcasted_bits=507.62Mibit, aggregated_bits=507.62Mibit
round  1, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.10029791), ('loss', 2.3102622), ('num_examples', 1007), ('num_batches', 55)]), broadcasted_bits=1015.24Mibit, aggregated_bits=1015.25Mibit
round  2, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.10710711), ('loss', 2.3048222), ('num_examples', 999), ('num_batches', 54)]), broadcasted_bits=1.49Gibit, aggregated_bits=1.49Gibit
round  3, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.1061061), ('loss', 2.3066027), ('num_examples', 999), ('num_batches', 55)]), broadcasted_bits=1.98Gibit, aggregated_bits=1.98Gibit
round  4, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.1287594), ('loss', 2.2999024), ('num_examples', 1064), ('num_batches', 58)]), broadcasted_bits=2.48Gibit, a

Inicialize o TensorBoard com o diretório de logs raiz especificado acima para exibir as métricas de treinamento. Pode demorar alguns segundos para os dados serem carregados. Exceto pela perda e exatidão, geramos como saída a quantidade de dados transmitidos via broadcast e agregados. Os dados transmitidos via broadcast referem-se aos tensores que o servidor envia para cada cliente, enquanto os dados agregados referem-se aos tensores que cada cliente envia de volta ao servidor.

In [ ]:
#@test {"skip": true}
%tensorboard --logdir /tmp/logs/scalars/ --port=0

## Crie uma função de agregação personalizada

Agora, vamos implementar uma função para usar algoritmos de compressão com perda nos dados agregados. Para isso, usaremos a API do TFF para criar uma `tff.aggregators.AggregationFactory`. Embora os pesquisadores costumem querer implementar seu próprio agregador (o que pode ser feito usando a API `tff.aggregators`), usaremos um método integrado para isso, especificamente `tff.learning.compression_aggregator`.

É importante observar que esse agregador não aplica compressão a todo o modelo de uma só vez. Em vez disso, aplica compressão somente às variáveis no modelo que são suficientemente grandes. De forma geral, variáveis pequenas, como bias, têm maior sensibilidade à inexatidão e, por serem relativamente pequenas, a possível economia de comunicação também é relativamente pequena.

In [ ]:
compression_aggregator = tff.learning.compression_aggregator()
isinstance(compression_aggregator, tff.aggregators.WeightedAggregationFactory)

True

Acima, podemos ver que o agregador de compressão é uma fábrica de agregação *com pesos*, ou seja, envolve agregação com pesos (ao contrário de agregadores para privacidade diferencial, que costumam não ter pesos).

A fábrica de agregação pode ser alimentada diretamente no FedAvg via seu argumento `model_aggregator`.

In [ ]:
federated_averaging_with_compression = tff.learning.algorithms.build_weighted_fed_avg(
    tff_model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0),
    model_aggregator=compression_aggregator)

## Treine o modelo novamente

Agora, vamos executar o novo algoritmo de cálculo federado de médias.

In [ ]:
logdir_for_compression = "/tmp/logs/scalars/compression/"
summary_writer_for_compression = tf.summary.create_file_writer(
    logdir_for_compression)

train(federated_averaging_process=federated_averaging_with_compression, 
      num_rounds=10,
      num_clients_per_round=10,
      summary_writer=summary_writer_for_compression)

round  0, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.087804876), ('loss', 2.3126457), ('num_examples', 1025), ('num_batches', 55)]), broadcasted_bits=507.62Mibit, aggregated_bits=146.47Mibit
round  1, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.073267326), ('loss', 2.3111901), ('num_examples', 1010), ('num_batches', 56)]), broadcasted_bits=1015.24Mibit, aggregated_bits=292.93Mibit
round  2, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.08925144), ('loss', 2.3071017), ('num_examples', 1042), ('num_batches', 57)]), broadcasted_bits=1.49Gibit, aggregated_bits=439.40Mibit
round  3, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.07985144), ('loss', 2.3061485), ('num_examples', 1077), ('num_batches', 59)]), broadcasted_bits=1.98Gibit, aggregated_bits=585.86Mibit
round  4, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.11947791), ('loss', 2.302166), ('num_examples', 996), ('num_batches', 55)]), broadcasted_bits=2.48

Inicialize o TensorBoard novamente para comparar as métricas de treinamento entre as duas execuções.

Como podemos ver no TensorBoard, há uma redução considerável entre a curva `original` e `compression` nos gráficos `aggregated_bits`, enquanto nos gráficos de `loss` e `sparse_categorical_accuracy`, as duas curvas são muito similares.

Concluindo, implementamos um algoritmo de compressão que tem desempenho similar ao do algoritmo original de cálculo federado de médias, mas com custo de comunicação consideravelmente menor.

In [ ]:
#@test {"skip": true}
%tensorboard --logdir /tmp/logs/scalars/ --port=0

## Exercícios

Para implementar um algoritmo de compressão personalizado e aplicá-lo ao loop de treinamento, você pode:

1. Implementar um novo algoritmo de compressão como subclasse de [tff.aggregators.MeanFactory](https://www.tensorflow.org/federated/api_docs/python/tff/aggregators/MeanFactory).
2. Fazer o treinamento com o algoritmo de compressão para ver se ele tem desempenho melhor do que o algoritmo acima.

Confira alguns possíveis campos para pesquisa: quantização não uniforme, compressão sem perda, como Codificação de Huffman, e mecanismos para adaptar a compressão com base nas informações de rodadas de treinamento anteriores.

Leituras recomendadas:

- [Expanding the Reach of Federated Learning by Reducing Client Resource Requirements](https://research.google/pubs/pub47774/) (Expansão do alcance do aprendizado federado por meio da redução dos requisitos de recursos dos clientes).
- [Federated Learning: Strategies for Improving Communication Efficiency](https://research.google/pubs/pub45648/) (Aprendizado federado: Estratégias para melhorar a eficiência da comunicação).
- *Seção 3.5 – Communication and Compression* (Comunicação e compressão) em [Advanced and Open Problems in Federated Learning](https://arxiv.org/abs/1912.04977) (Problemas avançados e em aberto no aprendizado federado).